<a href="https://colab.research.google.com/github/MarioCastilloM/AlgoTrading/blob/main/AlgoTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlgoTrading Python

## Libraries

In [1]:
!pip install ta
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas_datareader as pdr
from ta import add_all_ta_features
from ta.utils import dropna

  Created wheel for ta: filename=ta-0.7.0-py3-none-any.whl size=28716 sha256=c843b3d6cfd4991d4f455f034796a198c7f19920f12f7eccfc7687f96ba77b91
  Stored in directory: /root/.cache/pip/wheels/5e/74/e0/72395003bd1d3c8f3f5860c2d180ff15699e47a2733d8ebd38
Successfully built ta


In [2]:
tickers = ['btcusd', 'ethusd', 'eosusd']

In [3]:
crypto=pdr.get_data_tiingo(tickers, api_key = 'e03221a264e57841f1ff73d92bfcaff7008c4291')

In [4]:
crypto.reset_index(inplace=True)
crypto.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,btcusd,2016-08-11 00:00:00+00:00,590.27,605.512500,580.010000,597.884995,6186.622209,590.27,605.512500,580.010000,597.884995,6186.622209,0.0,1.0
1,btcusd,2016-08-12 00:00:00+00:00,585.22,603.210000,582.628634,589.010001,3289.468220,585.22,603.210000,582.628634,589.010001,3289.468220,0.0,1.0
2,btcusd,2016-08-13 00:00:00+00:00,585.91,592.019162,580.470000,587.046018,2484.341965,585.91,592.019162,580.470000,587.046018,2484.341965,0.0,1.0
3,btcusd,2016-08-14 00:00:00+00:00,583.52,589.980000,561.002903,585.613826,2032.613346,583.52,589.980000,561.002903,585.613826,2032.613346,0.0,1.0
4,btcusd,2016-08-15 00:00:00+00:00,568.76,598.824987,556.240000,567.990917,5567.695302,568.76,598.824987,556.240000,567.990917,5567.695302,0.0,1.0


In [5]:
btc = crypto[crypto['symbol']=='btcusd']

In [6]:
btc_ft = add_all_ta_features(btc[['date', 'open', 'high', 'low', 'close', 'volume']], open="open", high="high", low="low", close="close", volume="volume", fillna=True)

/usr/local/lib/python3.7/dist-packages/ta/wrapper.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high=df[high], low=df[low], close=df[close], volume=df[volume], fillna=fillna
/usr/local/lib/python3.7/dist-packages/ta/wrapper.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close=df[close], volume=df[volume], fillna=fillna
/usr/local/lib/python3.7/dist-packages/ta/wrapper.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [7]:
btc_ft

,date,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,...,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2016-08-11 00:00:00+00:00,597.884995,605.512500,580.010000,590.270000,6186.622209,-1.208699e+03,6.186622e+03,-0.195373,0.000000e+00,50.000000,0.000000e+00,0.000000e+00,-5755.051420,1000.000000,591.930833,0.000000,590.270000,590.270000,590.270000,0.000000,0.000000,0.0,0.0,591.930833,617.433333,566.428333,8.616716,0.467438,0.0,0.0,580.010000,605.512500,592.761250,4.320480,0.402313,0.000000,0.000000,0.000000,0.000000,...,0.000000,-94.614736,1.000000,0.000000,10802.018989,-948.186883,-948.186883,0.000000,592.761250,592.761250,592.761250,592.761250,11173.603880,11024.253312,4.0,4.0,0.0,-1.000000,-1.000000,0.0,0.0,0.00000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.231350,40.231350,-59.768650,0.000000,590.270000,0.000000,0.000000,0.000000,0.000000,-94.818688,0.000000,0.000000
1,2016-08-12 00:00:00+00:00,589.010001,603.210000,582.628634,585.220000,3289.468220,-3.669824e+03,2.897154e+03,-0.387272,-1.661181e+04,0.000000,9.889844e+04,9.889844e+04,-5894.216768,991.444593,591.383072,0.000000,587.745000,592.795000,582.695000,1.718432,0.250000,0.0,0.0,591.141856,614.183789,568.099923,7.795737,0.371498,0.0,0.0,580.010000,605.512500,592.761250,4.339042,0.204294,0.000000,-0.402849,-0.080570,-0.322279,...,0.000029,-0.001671,1.968885,-66.666667,10804.543989,-948.408524,-948.297704,-0.110821,592.761250,592.761250,592.761250,592.761250,11173.603880,11024.253312,4.0,8.0,-4.0,-1.000000,-1.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,-100.000000,5.623153,20.429370,30.330360,-79.570630,0.000000,588.535153,0.000000,-3.869915,-0.773983,-3.095932,-0.855541,-0.859221,-0.855541
2,2016-08-13 00:00:00+00:00,587.046018,592.019162,580.470000,585.910000,2484.341965,-3.813768e+03,5.381496e+03,-0.318865,-1.399381e+04,0.000000,-3.102939e+06,-1.502020e+06,-25.213591,992.613550,590.292573,0.000000,587.133333,591.604883,582.661784,1.523180,0.363209,0.0,0.0,589.472255,608.683264,570.261246,6.518037,0.407286,0.0,0.0,580.010000,605.512500,592.761250,4.343562,0.231350,0.000000,-0.658838,-0.196223,-0.462614,...,-0.001202,-0.004158,2.859059,-100.000000,10805.155655,-948.462216,-948.352541,-0.109675,592.761250,592.761250,592.761250,592.761250,11173.603880,11024.253312,4.0,8.0,-4.0,-1.000000,605.512500,0.0,1.0,0.00000,12.826982,0.000000,0.000000,0.000000,-99.696830,13.935354,23.134987,27.931902,-76.865013,0.000000,587.662051,0.000000,-8.291962,-2.277579,-6.014383,0.117904,0.117835,-0.738645
3,2016-08-14 00:00:00+00:00,585.613826,589.980000,561.002903,583.520000,2032.613346,-2.687434e+03,3.348883e+03,-0.192055,-1.268869e+04,0.000000,-1.532975e+07,-6.111262e+06,-5.362136,988.564555,588.531319,0.000000,586.230000,591.208775,581.251225,1.698574,0.227845,0.0,0.0,586.646100,608.298631,564.993569,7.381804,0.427812,0.0,0.0,561.002903,605.512500,583.257701,7.592514,0.505893,0.000000,-1.042546,-0.365488,-0.677058,...,-0.003177

In [8]:
import plotly.express as px

fig = px.line(btc_ft, x = btc_ft['date'], y = btc_ft.iloc[:,1], title = 'Technical Analysis')
for col in btc_ft.columns[2:]:
  if 'volume' not in col and 'volatility' not in col and 'momentum' not in col and 'others' not in col:
    fig.add_scatter(x = btc_ft['date'], y = btc_ft[col], name = col)
fig.show()

In [19]:
def ema(ts, days, smooth):
  ts = np.array(ts)
  ma = np.zeros(len(ts))
  ma[0] = ts[0]
  for i in range(1, len(ts)):
    ma[i] = ts[i]*(smooth/(1+days)) + ma[i-1]*(1-(smooth/(1+days)))
  return ma

In [47]:
ema_btc = ema(btc.close, 26, 2)
ema_btc_12 = ema(btc.close, 12, 2)
MACD = ema_btc - ema_btc_12
signal = ema(MACD, 9, 2)
histogram = MACD-signal

In [73]:
import plotly.subplots as sp

# Create figures in Express
figure1 = px.line(btc_ft, x = 'date', y = 'close', title = 'BTC MACD vs Signal Line')

fig = px.line(x = btc.date, y = MACD,
              labels={
                     "x": "Date",
                     "y": ""
                     })
# fig.add_scatter(x = btc.date, y = ema_btc_12, name = 'EMA(12)')
# fig.add_scatter(x = btc.date, y = ema_btc, name = 'EMA(26)')
# fig.add_scatter(x = btc.date, y = MACD, name = 'MACD')
fig = fig.add_scatter(x = btc.date, y = signal, name = 'Signal')
fig = fig.add_bar(x = btc.date, y = histogram, name = 'Histograma')

figure2 = fig

# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []
for trace in range(len(figure1["data"])):
    figure1_traces.append(figure1["data"][trace])
for trace in range(len(figure2["data"])):
    figure2_traces.append(figure2["data"][trace])

#Create a 1x2 subplot
this_figure = sp.make_subplots(rows=2, cols=1) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    this_figure.append_trace(traces, row=2, col=1)

#the subplot as shown in the above image
this_figure

In [74]:
# fig = px.line(x = btc.date, y = MACD, title = 'BTC MACD vs Signal Line',
#               labels={
#                      "x": "Date",
#                      "y": ""
#                      })
# fig.add_scatter(x = btc.date, y = ema_btc_12, name = 'EMA(12)')
# fig.add_scatter(x = btc.date, y = ema_btc, name = 'EMA(26)')
# fig.add_scatter(x = btc.date, y = MACD, name = 'MACD')
# fig.add_scatter(x = btc.date, y = signal, name = 'Signal')
# fig.add_bar(x = btc.date, y = histogram, name = 'Histograma')
